In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp
from functools import partial
import seaborn as sns
import requests
from data_loading import *
import plotly.graph_objects as go
import os
import os.path as osp
import re

In [ ]:
# use this to keep the cells wider, very nice on wide screens, set the percentage as you like it
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
conf = {
    'font.size': 14.0,
    'axes.grid': True,
    'axes.axisbelow': True,
    'axes.edgecolor': 'black',
#    'axes.facecolor': '#E5E5EF',
#    'axes.facecolor': '#E0E0EF',
    'axes.labelcolor': 'black',
    'axes.titlesize': 15.6,
    'axes.labelsize': 'large',    
    'figure.figsize': (12, 7),
    'figure.titlesize': 'x-large',
    'grid.linewidth': 1.3,
    'xtick.labelsize': 'large',
    'ytick.labelsize': 'large',
    'xtick.color': 'black',
    'ytick.color': 'black',
    'legend.fontsize': 'large',
    'figure.figsize': (12., 8.)
}

plt.rcParams.update(conf)

### loading cleaned data

In [ ]:
df_form, columns_series, program_columns_ratings, program_columns, form_ratings, form_attends, open_answers = load_form_data()
df_app = load_app_data()
df_timetable = load_api_info(df_form.columns)
app_ratings = prepare_app_ratings(df_app)
program_ratings, program_attends, program_columns = merge_ratings(form_ratings, app_ratings)
program_fun_score, program_info_score = calc_scores(program_ratings, program_columns)

### moderator feedback analysis

In [ ]:
df_form.columns[df_form.columns.to_series().str.contains('moder')]

In [ ]:
program_with_moderators = ['Animekvíz', 'Festdance', 'Cosplay debut', 'Cosplay soutěž', 'Sčítání žije!', 
                           'Festovní koncert', 'Soutěžní AMV', 'Alternativní móda: Od ponožky po klobouk']
# todo: in cleaning, check if some rows are completely duplicit in form data

In [ ]:
# some textual notes in 
# S čím jste byli letos nejvíce spokojeni / co se vám nejvíce líbilo?
# Co vás letos nejvíc zklamalo / nelíbilo se vám?


## rendering all program to pdf

In [ ]:
from jinja2 import Environment, PackageLoader, select_autoescape, FileSystemLoader
from PIL import Image

def generate_pdf():
    pd.set_option('display.max_colwidth', -1)
    moderator_comments = df_form['Jak jste byli spokojeni s moderátory soutěží a oficiálních pořadů?']
    comments_html = moderator_comments[moderator_comments != ''].to_frame(name='komentáře').to_html(index=False)
    
    per_lecture_comments = {}
    for column in program_with_moderators:
        comment_column = 'Komentář: ' + column
        comments_ser = df_form[df_form[comment_column].str.lower().str.contains('moder')][comment_column]
        comments_ser = comments_ser.append(df_app[(df_app['název'] == column) & df_app['komentář'].str.lower().str.contains('moder')]['komentář'])
        # todo: spojit s komentáři z appky
        per_lecture_comments[column] = comments_ser.to_frame(name='komentáře').to_html(index=False)
        
    data_root = osp.join(os.getcwd())
    pdf_path = data_root
    os.makedirs(pdf_path, exist_ok=True)
    
    env = Environment(loader=FileSystemLoader(os.getcwd()), trim_blocks=True)
    template = env.get_template('moderators-template.jinja2')
    html_report_path = osp.join(pdf_path, 'moderators-report.html')
    template.stream(per_lecture_comments=per_lecture_comments,
                    comments_html=comments_html
                   ).dump(html_report_path)

    chrome_exe = '"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"'
    pdf_file = osp.join(pdf_path, 'moderators-report.pdf')
    command = f'{chrome_exe} --headless --disable-gpu --no-sandbox --print-to-pdf="{pdf_file}" "{html_report_path}"'
    os.popen(command)

generate_pdf()


In [ ]:
df_form.columns.to_series()[df_form.columns.to_series().str.contains('kvíz')]